# LFM 梯度下降算法实现

### 0. 引入依赖


In [1]:
import numpy as np
import pandas as pd

### 1.数据准备

In [2]:
#评分矩阵R

R=np.array([[4,0,2,0,1],
    [0,2,3,0,0],
    [1,0,2,4,0],
    [5,0,0,3,1],
    [0,0,1,5,1],
    [0,3,2,4,1]])

R.shape


(6, 5)

### 2. 算法实现

梯度下降算法

$$
P_u:=P_u-\alpha \cdot \frac{\partial L}{\partial P_u}=P_u-\alpha \cdot\left[\sum_i 2\left(P_u^T Q_i-R_{u i}\right) Q_i+2 \lambda P_u\right]
$$

$$
Q_i:=Q_i-\alpha \cdot \frac{\partial L}{\partial Q_i}=Q_i-\alpha \cdot\left[\sum_u 2\left(P_u^T Q_i-R_{u i}\right) P_u+2 \lambda Q_i\right]
$$

In [3]:
"""
@输入参数：
R: M*N 的评分矩阵
K: 隐特征向量维度
max_iter: 最大迭代次数
alpha: 步长
lamda: 正则化系数

@输出
分解之后的P, Q
P: 初始化用户特征矩阵M*K
Q: 初始化物品特征矩阵N*K
"""

#给定超参数

K=7
max_iter = 5000
alpha = 0.0002
lamda=0.004

#核心算法
def LFM_grad_desc(R,max_iter,lamda, K=2,alpha=0.0001):
    #基本维度参数定义
    M=len(R)
    N=len(R[0])

    #P,Q初始值随机生成
    P=np.random.rand(M,K)
    Q=np.random.rand(N,K)
    Q=Q.T
    
    #开始迭代
    for steps in range(max_iter):
        #对所有的用户u物品i做遍历，对应的特征向量Pu，Qi梯度下降
        for u in range (M):
            for i in range(N):
                #对于每一个大于0的评分，求出预测评分误差
                if R[u][i]>0:
                    eui=np.dot(P[u],Q[:,i])-R[u][i]

                    #代入公式，按照梯度下降算法更新当前Pu和Qi
                    for k in range(K):
                        P[u][k]-=alpha*(2*eui*Q[k][i]+2*lamda*P[u][k]) ##这里对i的求和可以无视掉，因为我们已经对i进行了遍历
                        Q[k][i]-=alpha*(2*eui*P[u][k]+2*lamda*Q[k][i]) ##同样对u的求和可以无视因为对u也进行了遍历

    ##u,i遍历完成，所有特征向量更新完成，可以得到P，Q，可以计算预测评分矩阵
        predR=np.matmul(P,Q)

        ##计算当前损失函数
        cost=0
        for u in range(M):
            for i in range(N):
                if R[u][i] > 0:
                    cost += (np.dot(P[u,:],Q[:,i])-R[u][i]) ** 2
                    #加上正则化项
                    for k in range(K):
                        cost+=lamda * (P[u][k] ** 2 + Q[k][i] ** 2 )
        
        if cost < 0.0001:
            break




    return P,Q.T, cost

### 3.测试

In [4]:
P,Q,cost = LFM_grad_desc(R,max_iter,lamda, K,alpha)

predR=P.dot(Q.T)

print(cost)
print(R)

predR

0.5787042882401734
[[4 0 2 0 1]
 [0 2 3 0 0]
 [1 0 2 4 0]
 [5 0 0 3 1]
 [0 0 1 5 1]
 [0 3 2 4 1]]


array([[3.98795551, 2.10726749, 1.9991871 , 3.43746482, 1.0181826 ],
       [3.30091335, 2.09562135, 2.92014871, 3.25623383, 1.05565739],
       [1.00520644, 2.08016562, 1.99566042, 3.98946945, 0.81658078],
       [4.99607331, 2.81136464, 2.29543111, 2.99770168, 0.97034359],
       [3.19828934, 2.57326978, 1.00496311, 4.97707124, 1.02387099],
       [4.05099881, 2.89961788, 2.05055987, 4.02328965, 0.98618654]])